In [1]:
import cv2
import numpy as np

In [2]:
yolo_net = cv2.dnn.readNet("Path to yolo config", "Path to yolo weights")

In [3]:
with open("Path to coco.names") as f:
    classes = [line.strip() for line in f.readlines()]

In [1]:
# classes

In [2]:
# yolo_net.getLayerNames()

In [6]:
output_layers = yolo_net.getUnconnectedOutLayersNames()
output_layers

('yolo_82', 'yolo_94', 'yolo_106')

In [7]:
image = cv2.imread("Path to kite.jpg") #you could use any other picture but you'd have to tune the parameters to fit the picture. 

In [8]:
cv2.imshow('image', image)
cv2.waitKey(0)

-1

In [15]:
image = cv2.resize(image, (416, 416))

In [18]:
blob = cv2.dnn.blobFromImage(image, 1/255, (416, 416), swapRB = True, crop = False)

In [19]:
blob.shape

(1, 3, 416, 416)

In [20]:
yolo_net.setInput(blob)

In [21]:
outputs = yolo_net.forward(output_layers)

In [22]:
print(type(outputs))

<class 'tuple'>


In [23]:
height, widht, _ = image.shape
boxes = []
class_ids = []
confidences = []
for output in outputs:
    for detection in output:
        scores = detection[5:]
        confidence = float(np.max(scores))
        if confidence > 0.6:
            class_id = int(np.argmax(scores))
            
            cx = detection[0] * widht
            cy = detection[1] * height
            
            w = int(detection[2] * widht)
            h = int(detection[3] * height)
            
            x = int(cx - w/2)
            y = int(cy - h/2)
            
            boxes.append((x, y, w, h))
            confidences.append(confidence)
            class_ids.append(class_id)

In [24]:
len(boxes)

15

In [25]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)

In [26]:
indices

array([ 1,  3,  4,  8,  9, 13, 14,  7, 10, 12,  6])

In [27]:
len(indices)

11

In [28]:
for index in indices:
    x,y,w,h = boxes[index]
    confidence = confidences[index]
    label = classes[class_ids[index]]
    image = cv2.rectangle(image, (x,y), (x+w, y+h), (255,54,45), 1)
    image = cv2.putText(image, str(label) + ':' + str(round(confidence, 2)), (x, y-2), cv2.FONT_ITALIC, 1, (255,54,45), 1)

In [29]:
cv2.imshow('image', image)
cv2.waitKey(0)

-1